In [8]:
# Instalar paquetes
!pip install hubspot-api-client
!pip install rut_chile

In [ ]:
# ===================================================================
# CELDA 1: Configuración Inicial, Imports y Funciones
# ===================================================================
import pandas as pd
import re
from hubspot import HubSpot
from hubspot.crm.properties import ApiException, PropertyUpdate, Option
from rut_chile import rut_chile as rc
import requests
import numpy as np
import unicodedata
from datetime import datetime, timedelta
import time
import psutil
import os
import concurrent.futures
import sys

# --- Archivo de Log ---
# Se abre en modo "write" (w) para que se cree uno nuevo en cada ejecución.
log_file = open("log.txt", "w")
log_file.write("====================================================\n")
log_file.write("== INICIO DEL SCRIPT: Nuevos Carga Superintendencia ==\n")
log_file.write("====================================================\n")

start_time = time.time()


In [10]:
# Funciones para limpieza y manejo de datos

# Función para agregar un guion antes del último carácter en cada valor
def agregar_guion(valor):
    if pd.notna(valor) and len(valor) > 1:
        return valor[:-1] + '-' + valor[-1]
    else:
        return valor
# Función para eliminar 0 al principio de una cadena
def eliminar_cero(valor):
    if isinstance(valor, str) and valor.startswith('0'):
        return valor[1:]
    return valor

# Función para validar rut
def validar_rut(rut):
    try:
        return rc.is_valid_rut(rut)
    except Exception as e:
        return False

# Función para validar email
def es_correo_valido(email):
    # Verificar si el email es NaN o vacío
    if pd.isna(email) or email.strip() == '':
        return False

    # Patrón de la expresión regular para un email válido
    patron = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

    # Usar re.match para comprobar si el email coincide con el patrón
    if re.match(patron, email):
        return True
    else:
        return False

# Función para formatear  y limpiar el número de celular
def limpiar_y_formatear_valor(celular):
    # Limpiar el valor dejando solo números
    celular = re.sub(r'\D', '', str(celular))

    # Función para formatear el número de celular
    if pd.isna(celular):
        return ''
    if len(celular) > 7:
        return f"+56-9-{celular[-8:-4]}-{celular[-4:]}"
    else:
        return celular

def validar_nombre(nombre):
    """
    Valida si un nombre es válido según ciertos criterios:
    - No esté vacío.
    - Contenga solo letras, apóstrofes y guiones.
    - No contenga números ni caracteres especiales no permitidos.
    - Longitud mínima de 5 caracteres y máxima de 80 caracteres.
    """
    # Definir el patrón de regex para nombres válidos
    patron = re.compile(r"^[A-Za-zÁÉÍÓÚáéíóúÑñÄäÖöÜü\s'-]+$")

    # Verificar si el nombre no cumple con alguno de los criterios
    if nombre.startswith('Hola') or nombre.startswith('HOLA') or not patron.match(nombre) or not (5 <= len(nombre) <= 80):
        return False
    return True

# Función para limpiar espacios
def limpiar_espacios(valor):
    if isinstance(valor, str):
        return ' '.join(valor.split()).strip()
    return valor

# Función para llenar valores NaN y vacíos
def llenar_email(row):
    if pd.notna(row['MailCorrespondencia']) and row['MailCorrespondencia'].strip() != '':
        return row['MailCorrespondencia']
    elif pd.notna(row['MailRecidencia']) and row['MailRecidencia'].strip() != '':
        return row['MailRecidencia']
    elif pd.notna(row['MailTrabajo1']) and row['MailTrabajo1'].strip() != '':
        return row['MailTrabajo1']
    elif pd.notna(row['MailTrabajo2']) and row['MailTrabajo2'].strip() != '':
        return row['MailTrabajo2']
    else:
        return np.nan

def validar_rcm(rcm):
    if 100 <= rcm <= 300000:
        return True
    else:
        return False

def limpiar_valor(valor):
    """Elimina caracteres especiales, convierte a minúsculas, y elimina tildes"""
    # Eliminar tildes y acentos
    valor_sin_tildes = unicodedata.normalize('NFKD', valor).encode('ASCII', 'ignore').decode('ASCII')
    # Eliminar caracteres especiales
    valor_limpio = re.sub(r'[^\w\s]', '', valor_sin_tildes)
    return valor_limpio.replace(' ', '_').lower()



API HubSpot

In [ ]:
log_file.write("\n--- Bloque 2: Extrayendo datos de HubSpot ---\n")
try:
    log_file.write("Inicializando cliente de HubSpot...\n")
    # CORREGIDO: Se usa la variable de entorno para el token
    api_client = HubSpot(access_token=os.getenv('HUBSPOT_API_KEY'))
    log_file.write("Cliente de HubSpot inicializado.\n")

    properties = [ 'address','firstname','segundo_nombre', 'lastname','apellido_materno', 'condicion', 'createdate','crs',
                'date_of_birth','direccion_de_trabajo','edad','email','estado_pago','fecha_inscripcion_al_colegio',
    'fecha_titulo','gender','graduation_date','mobilephone','rcm','rut', 'lastmodifieddate', 'lifecyclestage',
                  'phone','hs_object_source_label','especialidades','fecha_act_super']

    log_file.write(f"Recuperando contactos con {len(properties)} propiedades...\n")
    all_contacts = api_client.crm.contacts.get_all(properties=properties)
    contacts_list = [contact.to_dict() for contact in all_contacts]
    log_file.write(f"Se recuperaron {len(contacts_list)} contactos.\n")

    df_contacts = pd.DataFrame(contacts_list)
    df_col_dict = pd.json_normalize(df_contacts['properties'])
    df_final_contactos = df_contacts.drop(columns=['properties']).join(df_col_dict)
    df_final_contactos = df_final_contactos.sort_values(by=['createdate'], ascending = True)
    log_file.write("DataFrames de HubSpot creados y procesados.\n")

except ApiException as e:
    log_file.write(f"!!! ERROR al conectar con HubSpot: {e}\n")

In [ ]:
log_file.write("\n--- Bloque 3: Filtrando RUTs para consulta externa ---\n")
# Manejo de dataframe para busqueda de información en la SUPER

# crear columna FullName
columnas = ['firstname','segundo_nombre', 'lastname','apellido_materno']
# Reemplazar valores nulos por cadenas vacías en las columnas especificadas
df_final_contactos[columnas] = df_final_contactos[columnas].fillna('')
# Reemplazar ` por ' en las columnas especificadas
df_final_contactos[columnas] = df_final_contactos[columnas].replace('`', "'", regex=True)
df_final_contactos[columnas] = df_final_contactos[columnas].replace('´', "'", regex=True)
df_final_contactos['full_name'] = df_final_contactos['firstname'].astype(str) +' '+ df_final_contactos['segundo_nombre'].astype(str)+' '+ df_final_contactos['lastname'].astype(str) + ' ' + df_final_contactos['apellido_materno'].astype(str)
# Eliminar dobles (o múltiples) espacios en la columna 'full name'
df_final_contactos['full_name'] = df_final_contactos['full_name'].str.replace(r'\s+', ' ', regex=True)
# Transformar a mayúsculas la columna 'full_name'
df_final_contactos['full_name'] = df_final_contactos['full_name'].str.upper()
# Aplicar la función nombre_valido y crear una nueva columna con los resultados
df_final_contactos['fullname_valido'] = df_final_contactos['full_name'].apply(validar_nombre)
# Agregar una nueva columna que marque si 'full_name' es nula o vacía
df_final_contactos['fullname_nulo_o_vacio'] = df_final_contactos['full_name'].isna() | (df_final_contactos['full_name'] == '')
# crear columna con analisis de duplicados segun nombre
df_final_contactos['fullname_duplicado'] = df_final_contactos.duplicated(subset=['full_name'], keep='first')


# Función para determinar el valor de 'rut_final' basado en las reglas dadas
def determinar_fullname(row):
    if row['fullname_nulo_o_vacio']:
        return 'Nulo'
    elif not row['fullname_valido']:
        return 'Invalido'
    elif row['fullname_duplicado']:
        return 'Duplicado'
    else:
        return 'Valido'

# Aplicar la función a cada fila del DataFrame
df_final_contactos['fullname_final'] = df_final_contactos.apply(determinar_fullname, axis=1)

#Trabajar variable RUT

# Agregar una nueva columna que marque si 'rut' es nula o vacía
df_final_contactos['rut_nulo_o_vacio'] = df_final_contactos['rut'].isna() | (df_final_contactos['rut'] == '')

# Reemplazar 'k' por 'K' en rut
df_final_contactos['rut'] = df_final_contactos['rut'].str.replace('k', 'K')

#normalizar columna rut
df_final_contactos['rut_normalizado'] = (
    df_final_contactos['rut'].str.replace(r'[\s.,-]', '', regex=True).apply(agregar_guion).apply(eliminar_cero).astype(str))

# Aplicar la función is_valid_rut y crear una nueva columna con los resultados
df_final_contactos['rut_valido'] = df_final_contactos['rut_normalizado'].apply(validar_rut)

# crear columna con analisis de duplicados segun rut para df_contactos
df_final_contactos['rut_duplicado'] = df_final_contactos.duplicated(subset=['rut_normalizado'], keep='first')

# Función para determinar el valor de 'rut_final' basado en las reglas dadas
def determinar_rut_final(row):
    if row['rut_nulo_o_vacio']:
        return 'Nulo'
    elif not row['rut_valido']:
        return 'Invalido'
    elif row['rut_duplicado']:
        return 'Duplicado'
    else:
        return 'Valido'

# Aplicar la función a cada fila del DataFrame
df_final_contactos['rut_final'] = df_final_contactos.apply(determinar_rut_final, axis=1)

#Trabajar variable EMAIL

# Agregar una nueva columna que marque si 'email' es nula o vacía
df_final_contactos['email_nulo_o_vacio'] = df_final_contactos['email'].isna() | (df_final_contactos['email'] == '')

# Aplicar la función de validación
df_final_contactos['email_valido'] = df_final_contactos['email'].apply(lambda x: es_correo_valido(x) if x else False)

# crear columna con analisis de duplicados segun email para df_contactos
df_final_contactos['email_duplicado'] = df_final_contactos.duplicated(subset=['email'], keep='first')

# Función para determinar el valor de 'email_final' basado en las reglas dadas
def determinar_email_final(row):
    if row['email_nulo_o_vacio']:
        return 'Nulo'
    elif not row['email_valido']:
        return 'Invalido'
    elif row['email_duplicado']:
        return 'Duplicado'
    else:
        return 'Valido'

# Aplicar la función a cada fila del DataFrame
df_final_contactos['email_final'] = df_final_contactos.apply(determinar_email_final, axis=1)

# Trabajar variable  RCM

# Convertir la columna 'rcm' a numérico y luego a int
df_final_contactos['rcm'] = pd.to_numeric(df_final_contactos['rcm'], errors='coerce')
df_final_contactos['rcm'] = df_final_contactos['rcm'].fillna(-1).astype(int)
df_final_contactos['rcm'] = df_final_contactos['rcm'].replace(-1, np.nan)

# Agregar una nueva columna que marque si 'RCM' es nula o vacía
df_final_contactos['rcm_nulo_o_vacio'] = df_final_contactos['rcm'].isna() | (df_final_contactos['rcm'] == '')

# Agregar una nueva columna que marque si 'RCM' es valida
df_final_contactos['rcm_valido'] = df_final_contactos['rcm'].apply(validar_rcm)

# crear columna con analisis de duplicados segun email para df_contactos
df_final_contactos['rcm_duplicado'] = df_final_contactos.duplicated(subset=['rcm'], keep='first')

# Función para determinar el valor de 'rcm_final' basado en las reglas dadas
def determinar_rcm_final(row):
    if row['rcm_nulo_o_vacio']:
        return 'Nulo'
    elif not row['rcm_valido']:
        return 'Invalido'
    elif row['rcm_duplicado']:
        return 'Duplicado'
    else:
        return 'Valido'

# Aplicar la función a cada fila del DataFrame
df_final_contactos['rcm_final'] = df_final_contactos.apply(determinar_rcm_final, axis=1)

#Trabajar variable fecha de nacimiento
df_final_contactos['DOB'] =  pd.to_datetime(df_final_contactos['date_of_birth'], errors='coerce')
# Verificar qué filas tienen fechas válidas
df_final_contactos['DOB_valida'] = df_final_contactos['DOB'].notna()

# Extraer el RUT sin el dígito verificador y agregarlo a una nueva columna usando .loc
df_final_contactos['rut_sdv'] = df_final_contactos['rut_normalizado'].str.split('-').str[0]
df_ruts = df_final_contactos[df_final_contactos['rut_valido'] == True]


In [13]:
# formatear fecha y definir conjunto de medios, con más 30 de días desde ultima actualización o fecha nula

# Asegurarse de que la columna 'fecha_act_super' esté en formato de fecha
df_ruts['fecha_act_super'] = pd.to_datetime(df_ruts['fecha_act_super'], format='%Y-%m-%d', errors='coerce')

# Filtrar los registros donde la fecha es nula o donde la fecha es anterior o igual a la fecha límite
df_filtrado_rut = df_ruts.loc[(df_ruts['fecha_act_super'].isna())]
df_filtrado_rut = df_filtrado_rut.sort_values(by='fecha_act_super')
# Obtener la lista de RUTs sin el dígito verificador
lista_ruts = df_filtrado_rut['rut_sdv'].unique().tolist()

len(lista_ruts)

/tmp/ipython-input-13-3857188973.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ruts['fecha_act_super'] = pd.to_datetime(df_ruts['fecha_act_super'], format='%Y-%m-%d', errors='coerce')


11629

API SuperIntendencia

In [ ]:
import concurrent.futures
import requests
import pandas as pd
import sys

log_file.write("\n--- Bloque 4: Consultando API de la Superintendencia ---\n")

url_base = "https://apis.superdesalud.gob.cl/api/prestadores/rut/"
# CORREGIDO: Se usa la variable de entorno para la apikey
apikey = os.getenv('SUPERDESALUD_API_KEY')
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Accept": "application/json"
}

def consultar_api_super(rut):
    url = f"{url_base}{rut}.json/?apikey={apikey}"
    try:
        response = requests.get(url, headers=headers, timeout=30)
        if response.status_code == 200:
            data = response.json()
            data["rut"] = rut
            return data
        else:
            return None
    except requests.RequestException as e:
        log_file.write(f"Error de conexión con RUT {rut}: {e}\n")
        return None

def obtener_datos_concurrente(ruts, max_workers=20):
    resultados = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(consultar_api_super, rut): rut for rut in ruts}
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            if (i + 1) % 20 == 0 or (i + 1) == len(ruts):
                log_file.write(f"Progreso de consulta: {i+1}/{len(ruts)} RUTs procesados\n")
            resultado = future.result()
            if resultado:
                resultados.append(resultado)
    return pd.DataFrame(resultados)

df_resultados = obtener_datos_concurrente(lista_ruts, max_workers=20)
log_file.write(f"Consulta a la Superintendencia finalizada. Se obtuvieron {len(df_resultados)} registros exitosos.\n")

Progreso: 1/11629 IDs procesados
Error con RUT 1948270: HTTPSConnectionPool(host='apis.superdesalud.gob.cl', port=443): Read timed out. (read timeout=30)
Error con RUT 1964637: HTTPSConnectionPool(host='apis.superdesalud.gob.cl', port=443): Read timed out. (read timeout=30)
Error con RUT 1700497: HTTPSConnectionPool(host='apis.superdesalud.gob.cl', port=443): Read timed out. (read timeout=30)
Error con RUT 1826280: HTTPSConnectionPool(host='apis.superdesalud.gob.cl', port=443): Read timed out. (read timeout=30)
Error con RUT 2244041: HTTPSConnectionPool(host='apis.superdesalud.gob.cl', port=443): Read timed out. (read timeout=30)
Error con RUT 2322670: HTTPSConnectionPool(host='apis.superdesalud.gob.cl', port=443): Read timed out. (read timeout=30)
Error con RUT 489976: HTTPSConnectionPool(host='apis.superdesalud.gob.cl', port=443): Read timed out. (read timeout=30)
Error con RUT 1842358: HTTPSConnectionPool(host='apis.superdesalud.gob.cl', port=443): Read timed out. (read timeout=30)


In [15]:
df_resultados

,nro_registro,sexo,nombres,apellido_paterno,apellido_materno,fecha_nacimiento,fecha_registro,nacionalidad,rut,dv,codigo_busqueda,universidad,observaciones,fecha_carga,antecedentes
0,362722,Femenino,Isabel Soledad,Fuentes,Zambrano,19/11/1988,02/06/2014,Chilena,17043756,K,Médico Cirujano,Universidad Católica,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede..."
1,676762,Femenino,Dennys Del Valle,Rodriguez,De Cepeda,No Informada,03/08/2021,Venezolana,27023628,6,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede..."
2,580765,Femenino,Ana Maria,Campos,Romero,11/08/1989,03/05/2019,Extranjera,26057534,1,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede..."
3,657127,Femenino,Teresita María,Fuenzalida,Navarro,20/09/1995,26/03/2021,Chilena,19243195,6,Médico Cirujano,Universidad del Desarrollo,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede..."
4,404629,Femenino,Katherine Juliet,Morales,Cortes,15/10/1989,22/03/2016,Extranjera,25184204,3,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,14070,Femenino,Ximena Loreto,Parada,Debia,10/11/1959,21/04/2009,Chilena,7880295,2,Médico Cirujano,Universidad de Concepción - Concepción,,11/07/2025,"[{'clase_antecedente': 'Especialidad', 'cod_an..."
65,614426,Femenino,Valentina,Lucchini,Wortzman,14/03/1994,20/01/2020,Chilena,18639617,0,Médico Cirujano,Universidad de Chile,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede..."
66,851755,Femenino,Gabriela Paola,Mendes,Lopez,07/09/1997,16/09/2024,Portuguesa,28438089,4,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede..."
67,83353,Masculino,Antonio Misael,De Avila,Romero,19/04/1970,17/02/2010,Extranjera,22917811,3,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede..."


In [ ]:
#definir lista de universidades desde el dataframe de Super


# ===================================================================
# CELDA 5: Procesamiento de datos de la Superintendencia y generación de opciones
# ===================================================================

log_file.write("\n--- Bloque 5: Procesando datos de la Superintendencia ---\n")

# Asumo que el DataFrame se llama df_resultados de la celda anterior.
# Si está vacío, se omitirá el procesamiento.
if not df_resultados.empty:
    # CORRECCIÓN: Se verifica que las columnas 'rut' y 'dv' existan antes de usarlas.
    if 'rut' in df_resultados.columns and 'dv' in df_resultados.columns:
        log_file.write("Columnas 'rut' y 'dv' encontradas. Creando 'rut_normalizado'.\n")
        df_resultados['rut_normalizado'] = df_resultados['rut'].astype(str) + '-' + df_resultados['dv'].astype(str)

        # Extraer y procesar los antecedentes
        log_file.write("Procesando antecedentes para extraer especialidades y títulos...\n")
        antecedentes_data = df_resultados[['rut_normalizado', 'antecedentes']]
        antecedentes_desnormalizados = antecedentes_data.explode('antecedentes').reset_index(drop=True)
        
        # Prevenir error si la columna 'antecedentes' está vacía después de explode
        if 'antecedentes' in antecedentes_desnormalizados.columns and not antecedentes_desnormalizados['antecedentes'].isnull().all():
            antecedentes_desnormalizados = pd.concat(
                [antecedentes_desnormalizados.drop(['antecedentes'], axis=1), antecedentes_desnormalizados['antecedentes'].apply(pd.Series)],
                axis=1)

            # Generar lista de especialidades
            especialidades = antecedentes_desnormalizados[antecedentes_desnormalizados['clase_antecedente'] == 'Especialidad']
            lista_especialidades = sorted(especialidades['cod_antecedente'].dropna().unique())
            log_file.write(f"Se encontraron {len(lista_especialidades)} especialidades únicas.\n")

            # Generar lista de títulos
            contactos_titulos = antecedentes_desnormalizados[
                (antecedentes_desnormalizados['cod_antecedente'].isin(['Médica Cirujana', 'Médico Cirujano'])) &
                (antecedentes_desnormalizados['procedencia'] != 'EUNACOM')]
            
            # Prevenir error si 'tipo_antecedente' no existe
            if 'tipo_antecedente' in contactos_titulos.columns:
                contactos_titulos_ordenados = contactos_titulos.sort_values(by=['rut_normalizado', 'tipo_antecedente'], ascending=[True, True])
            else:
                contactos_titulos_ordenados = contactos_titulos # Continuar sin ordenar por tipo_antecedente

            contactos_titulos_unicos = contactos_titulos_ordenados.drop_duplicates(subset='rut_normalizado', keep='first').copy()
            contactos_titulos_unicos['procedencia'] = contactos_titulos_unicos['procedencia'].str.split(' -').str[0]
            opciones_procedencia = sorted(contactos_titulos_unicos['procedencia'].dropna().unique())
            opciones_procedencia = [valor for valor in opciones_procedencia if valor != '']
            log_file.write(f"Se encontraron {len(opciones_procedencia)} universidades/procedencias de títulos únicas.\n")

        else:
            log_file.write("Columna 'antecedentes' vacía o con problemas. Se omitirá la generación de opciones de especialidad y título.\n")
            lista_especialidades = []
            opciones_procedencia = []

        # Generar lista de nacionalidades
        if 'nacionalidad' in df_resultados.columns:
            lista_nacionalidad = sorted(df_resultados['nacionalidad'].dropna().unique())
            log_file.write(f"Se encontraron {len(lista_nacionalidad)} nacionalidades únicas.\n")
        else:
            lista_nacionalidad = []
            log_file.write("Columna 'nacionalidad' no encontrada.\n")


        # Generar opciones para las propiedades de HubSpot
        opciones = [{"label": item, "value": limpiar_valor(item)} for item in lista_especialidades]
        opciones_procedencia_titulo = [{"label": item, "value": limpiar_valor(item)} for item in opciones_procedencia]
        opciones_nacionalidad = [{"label": item, "value": limpiar_valor(item)} for item in lista_nacionalidad if item]
        log_file.write("Listas de opciones para HubSpot generadas correctamente.\n")

    else:
        log_file.write("!!! ADVERTENCIA: Las columnas 'rut' y/o 'dv' no se encontraron en los datos de la Superintendencia. Se omitirá todo el procesamiento de este bloque.\n")
        # Se definen las listas como vacías para que el resto del script no falle
        opciones = []
        opciones_procedencia_titulo = []
        opciones_nacionalidad = []
else:
    log_file.write("El DataFrame de la Superintendencia está vacío. Se omite este bloque de procesamiento.\n")
    opciones = []
    opciones_procedencia_titulo = []
    opciones_nacionalidad = []

[{'label': 'Chilena', 'value': 'chilena'},
 {'label': 'Colombiana', 'value': 'colombiana'},
 {'label': 'Extranjera', 'value': 'extranjera'},
 {'label': 'Portuguesa', 'value': 'portuguesa'},
 {'label': 'Venezolana', 'value': 'venezolana'}]

**Actualizar opciones de especialidades en HubSpot**

In [ ]:
import os
from hubspot import HubSpot
from hubspot.crm.properties import ApiException, PropertyUpdate, Option

# --- Mensajes de Log para este bloque ---
log_file.write("\n--- Bloque: Actualizando opciones de la propiedad 'especialidades' ---\n")

# Asumo que la variable 'opciones' (una lista de diccionarios) ya existe de una celda anterior
nuevas_opciones = opciones
property_name = 'especialidades'

try:
    # CORREGIDO: Se usa la variable de entorno para la llave de API
    access_token = os.getenv('HUBSPOT_API_KEY')
    api_client = HubSpot(access_token=access_token)
    
    log_file.write(f"Obteniendo la configuración actual de la propiedad '{property_name}'...\n")
    properties_api = api_client.crm.properties
    property_data = properties_api.core_api.get_by_name(object_type='contacts', property_name=property_name)

    current_options = property_data.options if property_data.options else []
    valores_existentes = {option.value for option in current_options}
    
    opciones_anadidas = 0
    # Añadir nuevas opciones que no existan previamente
    for option in nuevas_opciones:
        if option['value'] not in valores_existentes:
            new_option = Option(label=option['label'], value=option['value'])
            current_options.append(new_option)
            valores_existentes.add(option['value'])
            # CORREGIDO: Se escribe en el log qué opción se está añadiendo
            log_file.write(f"  -> Nueva opción para agregar: {option['label']}\n")
            opciones_anadidas += 1

    if opciones_anadidas > 0:
        log_file.write(f"Se agregarán {opciones_anadidas} nuevas opciones. Actualizando propiedad...\n")
        update_data = PropertyUpdate(
            label=property_data.label,
            options=current_options,
            group_name=property_data.group_name,
            type=property_data.type,
            field_type=property_data.field_type
        )
        properties_api.core_api.update(object_type='contacts', property_name=property_name, property_update=update_data)
        log_file.write(f"Éxito: La propiedad '{property_name}' fue actualizada.\n")
    else:
        log_file.write("No se encontraron nuevas opciones para agregar. La propiedad ya está actualizada.\n")

except ApiException as e:
    log_file.write(f"!!! ERROR al conectar con la API de HubSpot: {e}\n")
except Exception as e:
    log_file.write(f"!!! ERROR inesperado: {e}\n")

Propiedad actualizada con éxito


**ACTUALIZAR OPCIONES DE PROCEDENCIA TITULO MEDICO/A**

In [ ]:
# Nombre de la propiedad
property_name = 'universidad_titulo'

# Nuevas opciones a agregar (asegúrate de que la variable opciones esté definida)
nuevas_opciones = opciones_procedencia_titulo

try:
    # Obtener la propiedad actual
    properties_api = api_client.crm.properties
    property_data = properties_api.core_api.get_by_name(object_type='contacts', property_name=property_name)

    # Obtener opciones actuales
    current_options = property_data.options if property_data.options else []

    # Crear un conjunto de valores existentes para evitar duplicados
    valores_existentes = {option.value for option in current_options}

    # Añadir nuevas opciones a las opciones existentes
    for option in nuevas_opciones:
        if option['value'] not in valores_existentes:
            new_option = Option(label=option['label'], value=option['value'])
            current_options.append(new_option)
            valores_existentes.add(option['value'])  # Actualizar el conjunto de valores existentes

    # Preparar datos para la actualización
    update_data = PropertyUpdate(
        label=property_data.label,
        options=current_options,
        group_name=property_data.group_name,
        type=property_data.type,
        field_type=property_data.field_type    )

    # Actualizar la propiedad
    properties_api.core_api.update(object_type='contacts', property_name=property_name, property_update=update_data)

    log_file.write("Propiedad actualizada con éxito en HubSpot.\n")

except ApiException as e:
    log_file.write(f"Error al conectar con la API de HubSpot: {e}.\n")
except Exception as e:
    log_file.write(f"Error inesperado: {e}.\n")

Propiedad actualizada con éxito


In [ ]:
# Nombre de la propiedad
property_name = 'nacionalidad'

# Nuevas opciones a agregar (asegúrate de que la variable opciones esté definida)
nuevas_opciones = opciones_nacionalidad

try:
    # Obtener la propiedad actual
    properties_api = api_client.crm.properties
    property_data = properties_api.core_api.get_by_name(object_type='contacts', property_name=property_name)

    # Obtener opciones actuales
    current_options = property_data.options if property_data.options else []

    # Crear un conjunto de valores existentes para evitar duplicados
    valores_existentes = {option.value for option in current_options}

    # Añadir nuevas opciones a las opciones existentes
    for option in nuevas_opciones:
        if option['value'] not in valores_existentes:
            new_option = Option(label=option['label'], value=option['value'])
            current_options.append(new_option)
            valores_existentes.add(option['value'])  # Actualizar el conjunto de valores existentes

    # Preparar datos para la actualización
    update_data = PropertyUpdate(
        label=property_data.label,
        options=current_options,
        group_name=property_data.group_name,
        type=property_data.type,
        field_type=property_data.field_type    )

    # Actualizar la propiedad
    properties_api.core_api.update(object_type='contacts', property_name=property_name, property_update=update_data)

    log_file.write("Propiedad actualizada con éxito en HubSpot.\n")

except ApiException as e:
    log_file.write(f"Error al conectar con la API de HubSpot: {e}.\n")
except Exception as e:
    log_file.write(f"Error inesperado: {e}.\n")

Propiedad actualizada con éxito


In [20]:
# Cruzar los DataFrames por la columna 'rut_normalizado'
df_especialidades = pd.merge(especialidades, df_ruts, on='rut_normalizado', how='inner')
columnas = ['id','rut_normalizado','cod_antecedente','fecha_antecedente','procedencia']

df_especialidades = df_especialidades[columnas]
df_especialidades = df_especialidades.sort_values(by='fecha_antecedente', ascending=False)
# Eliminar duplicados según 'id' y 'cod_antecedente', manteniendo el primer registro (que es el más reciente)
df_especialidades_unicas= df_especialidades.drop_duplicates(subset=['id', 'cod_antecedente'], keep='first')
df_especialidades_unicas
# Aplica la función a la columna 'columna_a_limpiar'
df_especialidades_unicas['valor'] = df_especialidades_unicas['cod_antecedente'].apply(limpiar_valor)
columnas = ['id','valor']
df_especialidades_unicas = df_especialidades_unicas[columnas]
#agrupadas por id
especilidades_agrupados_id = df_especialidades_unicas.groupby('id')['valor'].apply(list).reset_index()
# Convertir el DataFrame en la lista de diccionarios
contactos_especialidad = especilidades_agrupados_id.rename(columns={'valor': 'nuevas_especialidades'}).to_dict(orient='records')
contactos_especialidad


/tmp/ipython-input-20-2789074610.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_especialidades_unicas['valor'] = df_especialidades_unicas['cod_antecedente'].apply(limpiar_valor)


[{'id': '103739051046',
  'nuevas_especialidades': ['traumatologia_y_ortopedia']},
 {'id': '108325892192', 'nuevas_especialidades': ['hematologia', 'pediatria']},
 {'id': '110983091472',
  'nuevas_especialidades': ['cirugia_de_cabeza_y_cuello_y_maxilofacial',
   'cirugia_general']},
 {'id': '119875443550',
  'nuevas_especialidades': ['obstetricia_y_ginecologia',
   'medicina_materno_fetal']},
 {'id': '13230',
  'nuevas_especialidades': ['psiquiatria_pediatrica_y_de_la_adolescencia',
   'psiquiatria_adulto']},
 {'id': '135957557859',
  'nuevas_especialidades': ['cirugia_general',
   'cirugia_plastica_y_reparadora']},
 {'id': '135974243000', 'nuevas_especialidades': ['imagenologia']},
 {'id': '136284681583',
  'nuevas_especialidades': ['obstetricia_y_ginecologia']},
 {'id': '136854337170', 'nuevas_especialidades': ['anestesiologia']},
 {'id': '137655262555', 'nuevas_especialidades': ['pediatria']},
 {'id': '137672059942',
  'nuevas_especialidades': ['psiquiatria_pediatrica_y_de_la_adoles

In [ ]:
import requests
import os

# --- Mensajes de Log para este bloque ---
log_file.write("\n--- Bloque: Actualizando Especialidades en HubSpot ---\n")

# CORREGIDO: Se usa la variable de entorno para la llave de API
ACCESS_TOKEN = os.getenv('HUBSPOT_API_KEY')
base_url = 'https://api.hubapi.com/crm/v3/objects/contacts'
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}',
    'Content-Type': 'application/json'
}

# Función para obtener la propiedad 'especialidades' de un contacto
def obtener_especialidades(contacto_id):
    url = f'{base_url}/{contacto_id}'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        especialidades = data.get('properties', {}).get('especialidades', '')
        return especialidades.split(';') if especialidades else []
    else:
        # CORREGIDO: Se escribe el error en el log
        log_file.write(f"!!! ERROR al obtener el contacto {contacto_id}: {response.status_code} - {response.text}\n")
        return []

# Función para actualizar la propiedad 'especialidades' de un contacto
def actualizar_especialidad(contacto_id, nuevas_especialidades):
    especialidades_existentes = obtener_especialidades(contacto_id)
    especialidades_a_agregar = [esp for esp in nuevas_especialidades if esp not in especialidades_existentes]

    if not especialidades_a_agregar:
        log_file.write(f"Info: Contacto {contacto_id} ya tiene todas las especialidades registradas. No se necesita actualización.\n")
        return True # Se considera un éxito porque no hay nada que hacer

    especialidades_combinadas = especialidades_existentes + especialidades_a_agregar
    especialidades_str = ';'.join(especialidades_combinadas)

    url = f'{base_url}/{contacto_id}'
    data = {'properties': {'especialidades': especialidades_str}}
    response = requests.patch(url, json=data, headers=headers)
    
    if response.status_code == 200:
        return True
    else:
        log_file.write(f"!!! ERROR al actualizar el contacto {contacto_id}: {response.status_code} - {response.text}\n")
        return False

# Asumo que la variable 'contactos_especialidad' ya existe de una celda anterior
contactos = contactos_especialidad
total_contactos = len(contactos)
log_file.write(f"Iniciando la actualización de especialidades para {total_contactos} contactos...\n")

# Iterar sobre la lista de contactos y actualizar la propiedad 'especialidades'
for index, contacto in enumerate(contactos):
    exito = actualizar_especialidad(contacto['id'], contacto['nuevas_especialidades'])
    if exito:
        log_file.write(f"({index + 1}/{total_contactos}) Éxito: Especialidades del contacto {contacto['id']} procesadas.\n")
    # El caso de error ya se registra dentro de la función

log_file.write("--- Proceso de actualización de especialidades finalizado ---\n")



Contacto 103739051046 actualizado exitosamente con las especialidades: traumatologia_y_ortopedia.
Contacto 108325892192 actualizado exitosamente con las especialidades: hematologia;pediatria.
Contacto 110983091472 actualizado exitosamente con las especialidades: cirugia_de_cabeza_y_cuello_y_maxilofacial;cirugia_general.
Contacto 119875443550 actualizado exitosamente con las especialidades: obstetricia_y_ginecologia;medicina_materno_fetal.
Contacto 13230 actualizado exitosamente con las especialidades: psiquiatria_pediatrica_y_de_la_adolescencia;psiquiatria_adulto.
Contacto 135957557859 actualizado exitosamente con las especialidades: cirugia_general;cirugia_plastica_y_reparadora.
Contacto 135974243000 actualizado exitosamente con las especialidades: imagenologia.
Contacto 136284681583 actualizado exitosamente con las especialidades: obstetricia_y_ginecologia.
Contacto 136854337170 actualizado exitosamente con las especialidades: anestesiologia.
Contacto 137655262555 actualizado exitosa

**ACTUALIZAR INFORMACIÓN DE UNIVERSIDAD DE ORIGEN**

In [22]:
df_titulo_medico = contactos_titulos_unicos [['rut_normalizado','fecha_antecedente','procedencia']]

df_titulo_medico['fecha_hoy'] = pd.to_datetime('today').strftime('%Y-%m-%d')

# Cruzar los DataFrames por la columna 'rut_normalizado'
df_titulo = pd.merge(df_titulo_medico, df_ruts, on='rut_normalizado', how='inner')
columnas = ['id','rut_normalizado','fecha_antecedente','procedencia','fecha_hoy']

df_titulo = df_titulo[columnas]
df_titulo = df_titulo.sort_values(by='fecha_antecedente', ascending=False)


# Aplica la función a la columna 'columna_a_limpiar'
df_titulo['cod_universidad'] = df_titulo['procedencia'].apply(limpiar_valor)
columnas = ['id','cod_universidad','fecha_antecedente','fecha_hoy']
df_titulo = df_titulo[columnas]

# Convertir la columna 'fecha_antecedente' al formato datetime
df_titulo['fecha_antecedente'] = pd.to_datetime(df_titulo['fecha_antecedente'], format='%d/%m/%Y')

# Convertir la columna 'fecha_antecedente' al formato 'YYYY-MM-DD'
df_titulo['fecha_antecedente'] = df_titulo['fecha_antecedente'].dt.strftime('%Y-%m-%d')

# Convertir el DataFrame en la lista de diccionarios
df_titulo = df_titulo.to_dict(orient='records')

# Lista de contactos con sus nuevos atributos
contactos = df_titulo

contactos

/tmp/ipython-input-22-45818152.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_titulo_medico['fecha_hoy'] = pd.to_datetime('today').strftime('%Y-%m-%d')


[{'id': '114248922891',
  'cod_universidad': 'pontificia_universidad_catolica_de_chile',
  'fecha_antecedente': '2020-12-31',
  'fecha_hoy': '2025-07-17'},
 {'id': '132679507091',
  'cod_universidad': 'pontificia_universidad_catolica_de_chile',
  'fecha_antecedente': '2020-12-31',
  'fecha_hoy': '2025-07-17'},
 {'id': '128074821424',
  'cod_universidad': 'pontificia_universidad_catolica_de_chile',
  'fecha_antecedente': '2019-12-31',
  'fecha_hoy': '2025-07-17'},
 {'id': '35516',
  'cod_universidad': 'pontificia_universidad_catolica_de_chile',
  'fecha_antecedente': '2019-12-31',
  'fecha_hoy': '2025-07-17'},
 {'id': '116454682802',
  'cod_universidad': 'universidad_del_desarrollo',
  'fecha_antecedente': '2022-12-30',
  'fecha_hoy': '2025-07-17'},
 {'id': '15661020',
  'cod_universidad': 'universidad_del_desarrollo',
  'fecha_antecedente': '2022-12-30',
  'fecha_hoy': '2025-07-17'},
 {'id': '136504770733',
  'cod_universidad': 'universidad_de_chile',
  'fecha_antecedente': '2003-12-30

In [ ]:
log_file.write("\n--- Bloque: Actualizando Universidad y Fecha de Título en HubSpot ---\n")

# Función para actualizar los atributos de un contacto
def actualizar_contacto(contacto_id, cod_universidad, fecha_antecedente, fecha_hoy):
    url = f'{base_url}/{contacto_id}'
    data = {
        'properties': {
            'universidad_titulo': cod_universidad,
            'fecha_titulo': fecha_antecedente,
            'fecha_act_super': fecha_hoy
        }
    }
    # La variable 'headers' se usa desde el ámbito global
    response = requests.patch(url, json=data, headers=headers)
    return response

# Asumo que la variable 'contactos' es una lista de diccionarios que ya existe
total_contactos = len(contactos)
log_file.write(f"Iniciando la actualización de {total_contactos} contactos con datos de universidad...\n")

# Iterar sobre la lista de contactos y actualizar los atributos
for index, contacto in enumerate(contactos):
    response = actualizar_contacto(
        contacto['id'], 
        contacto['cod_universidad'], 
        contacto['fecha_antecedente'],
        contacto['fecha_hoy']
    )

    # Escribir el resultado de cada actualización en el log
    if response.status_code == 200:
        log_file.write(f"({index + 1}/{total_contactos}) Éxito: Contacto {contacto['id']} actualizado.\n")
    else:
        log_file.write(f"!!! ERROR al actualizar contacto {contacto['id']}: {response.status_code} - {response.text}\n")

log_file.write("--- Proceso de actualización de universidad finalizado ---\n")


Contacto 114248922891 actualizado exitosamente 
Contacto 132679507091 actualizado exitosamente 
Contacto 128074821424 actualizado exitosamente 
Contacto 35516 actualizado exitosamente 
Contacto 116454682802 actualizado exitosamente 
Contacto 15661020 actualizado exitosamente 
Contacto 136504770733 actualizado exitosamente 
Contacto 27857390431 actualizado exitosamente 
Contacto 28162051 actualizado exitosamente 
Contacto 137943909435 actualizado exitosamente 
Contacto 127681079128 actualizado exitosamente 
Contacto 132415507780 actualizado exitosamente 
Contacto 108915471647 actualizado exitosamente 
Contacto 103739051046 actualizado exitosamente 
Contacto 19375 actualizado exitosamente 
Contacto 127681549592 actualizado exitosamente 
Contacto 104241185966 actualizado exitosamente 
Contacto 127734499908 actualizado exitosamente 
Contacto 120153223183 actualizado exitosamente 
Contacto 108915619727 actualizado exitosamente 
Contacto 138798961986 actualizado exitosamente 
Contacto 297056

In [24]:
df_resultados

,nro_registro,sexo,nombres,apellido_paterno,apellido_materno,fecha_nacimiento,fecha_registro,nacionalidad,rut,dv,codigo_busqueda,universidad,observaciones,fecha_carga,antecedentes,rut_normalizado
0,362722,Femenino,Isabel Soledad,Fuentes,Zambrano,19/11/1988,02/06/2014,Chilena,17043756,K,Médico Cirujano,Universidad Católica,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede...",17043756-K
1,676762,Femenino,Dennys Del Valle,Rodriguez,De Cepeda,No Informada,03/08/2021,Venezolana,27023628,6,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede...",27023628-6
2,580765,Femenino,Ana Maria,Campos,Romero,11/08/1989,03/05/2019,Extranjera,26057534,1,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede...",26057534-1
3,657127,Femenino,Teresita María,Fuenzalida,Navarro,20/09/1995,26/03/2021,Chilena,19243195,6,Médico Cirujano,Universidad del Desarrollo,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede...",19243195-6
4,404629,Femenino,Katherine Juliet,Morales,Cortes,15/10/1989,22/03/2016,Extranjera,25184204,3,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede...",25184204-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,14070,Femenino,Ximena Loreto,Parada,Debia,10/11/1959,21/04/2009,Chilena,7880295,2,Médico Cirujano,Universidad de Concepción - Concepción,,11/07/2025,"[{'clase_antecedente': 'Especialidad', 'cod_an...",7880295-2
65,614426,Femenino,Valentina,Lucchini,Wortzman,14/03/1994,20/01/2020,Chilena,18639617,0,Médico Cirujano,Universidad de Chile,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede...",18639617-0
66,851755,Femenino,Gabriela Paola,Mendes,Lopez,07/09/1997,16/09/2024,Portuguesa,28438089,4,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede...",28438089-4
67,83353,Masculino,Antonio Misael,De Avila,Romero,19/04/1970,17/02/2010,Extranjera,22917811,3,Médico Cirujano,,,11/07/2025,"[{'clase_antecedente': 'Título', 'cod_antecede...",22917811-3


**Actualizar Fecha Nacimiento, regitro Super y Genero**

In [25]:
#Actualizar información personal

# Extraer la columna 'antecedentes' junto con la llave
antecedentes_personales = df_resultados[['rut_normalizado', 'fecha_nacimiento','sexo','nro_registro','nacionalidad','nombres','apellido_paterno',	'apellido_materno']]

antecedentes_personales.loc[antecedentes_personales['fecha_nacimiento'] == 'No Informada'] = np.nan

# Convertir la columna 'fecha_antecedente' al formato datetime
antecedentes_personales['fecha_nacimiento'] = pd.to_datetime(antecedentes_personales['fecha_nacimiento'], format='%d/%m/%Y')

# Convertir la columna 'fecha_nacimiento' al formato 'YYYY-MM-DD'
antecedentes_personales['fecha_nacimiento'] = antecedentes_personales['fecha_nacimiento'].dt.strftime('%Y-%m-%d')

# Cruzar los DataFrames por la columna 'rut_normalizado'
df_antecedentes_personales = pd.merge(antecedentes_personales, df_ruts, on='rut_normalizado', how='inner')

df_antecedentes_personales

/tmp/ipython-input-25-3786877891.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antecedentes_personales['fecha_nacimiento'] = pd.to_datetime(antecedentes_personales['fecha_nacimiento'], format='%d/%m/%Y')
/tmp/ipython-input-25-3786877891.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antecedentes_personales['fecha_nacimiento'] = antecedentes_personales['fecha_nacimiento'].dt.strftime('%Y-%m-%d')


,rut_normalizado,fecha_nacimiento,sexo,nro_registro,nacionalidad,nombres,apellido_paterno,apellido_materno_x,associations,created_at,...,email_valido,email_duplicado,email_final,rcm_nulo_o_vacio,rcm_valido,rcm_duplicado,rcm_final,DOB,DOB_valida,rut_sdv
0,17043756-K,1988-11-19,Femenino,362722,Chilena,Isabel Soledad,Fuentes,Zambrano,None,2022-11-08 18:52:17.262000+00:00,...,True,False,Valido,False,True,False,Valido,1988-11-19,True,17043756
1,17043756-K,1988-11-19,Femenino,362722,Chilena,Isabel Soledad,Fuentes,Zambrano,None,2023-03-24 20:55:58.798000+00:00,...,True,False,Valido,False,True,True,Duplicado,NaT,False,17043756
2,26057534-1,1989-08-11,Femenino,580765,Extranjera,Ana Maria,Campos,Romero,None,2023-06-06 14:57:37.585000+00:00,...,True,False,Valido,True,False,True,Nulo,NaT,False,26057534
3,19243195-6,1995-09-20,Femenino,657127,Chilena,Teresita María,Fuenzalida,Navarro,None,2023-10-26 15:27:00.001000+00:00,...,True,False,Valido,False,True,False,Valido,NaT,False,19243195
4,19243195-6,1995-09-20,Femenino,657127,Chilena,Teresita María,Fuenzalida,Navarro,None,2025-04-07 20:22:30.297000+00:00,...,True,False,Valido,False,True,True,Duplicado,1995-09-20,True,19243195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,28438089-4,1997-09-07,Femenino,851755,Portuguesa,Gabriela Paola,Mendes,Lopez,None,2025-07-15 20:53:07.541000+00:00,...,True,False,Valido,True,False,True,Nulo,NaT,False,28438089
97,22917811-3,1970-04-19,Masculino,83353,Extranjera,Antonio Misael,De Avila,Romero,None,2024-02-15 20:19:41.791000+00:00,...,False,True,Nulo,False,True,False,Valido,1970-04-19,True,22917811
98,22917811-3,1970-04-19,Masculino,83353,Extranjera,Antonio Misael,De Avila,Romero,None,2025-07-17 15:08:16.701000+00:00,...,True,False,Valido,False,True,True,Duplicado,NaT,False,22917811
99,18631264-3,1994-03-01,Masculino,710725,Chilena,Mauricio Fernando,Bastías,Cortés,None,2025-03-26 03:28:35.956000+00:00,...,False,True,Nulo,True,False,True,Nulo,1994-03-01,True,18631264


In [26]:
df_antecedentes_personales = df_antecedentes_personales.loc[:, ~df_antecedentes_personales.columns.duplicated()]

valores_unicos = df_antecedentes_personales['nacionalidad'].unique()
valores_unicos

array(['Chilena', 'Extranjera', 'Venezolana', 'Portuguesa'], dtype=object)

In [27]:
df_antecedentes_personales

,rut_normalizado,fecha_nacimiento,sexo,nro_registro,nacionalidad,nombres,apellido_paterno,apellido_materno_x,associations,created_at,...,email_valido,email_duplicado,email_final,rcm_nulo_o_vacio,rcm_valido,rcm_duplicado,rcm_final,DOB,DOB_valida,rut_sdv
0,17043756-K,1988-11-19,Femenino,362722,Chilena,Isabel Soledad,Fuentes,Zambrano,None,2022-11-08 18:52:17.262000+00:00,...,True,False,Valido,False,True,False,Valido,1988-11-19,True,17043756
1,17043756-K,1988-11-19,Femenino,362722,Chilena,Isabel Soledad,Fuentes,Zambrano,None,2023-03-24 20:55:58.798000+00:00,...,True,False,Valido,False,True,True,Duplicado,NaT,False,17043756
2,26057534-1,1989-08-11,Femenino,580765,Extranjera,Ana Maria,Campos,Romero,None,2023-06-06 14:57:37.585000+00:00,...,True,False,Valido,True,False,True,Nulo,NaT,False,26057534
3,19243195-6,1995-09-20,Femenino,657127,Chilena,Teresita María,Fuenzalida,Navarro,None,2023-10-26 15:27:00.001000+00:00,...,True,False,Valido,False,True,False,Valido,NaT,False,19243195
4,19243195-6,1995-09-20,Femenino,657127,Chilena,Teresita María,Fuenzalida,Navarro,None,2025-04-07 20:22:30.297000+00:00,...,True,False,Valido,False,True,True,Duplicado,1995-09-20,True,19243195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,28438089-4,1997-09-07,Femenino,851755,Portuguesa,Gabriela Paola,Mendes,Lopez,None,2025-07-15 20:53:07.541000+00:00,...,True,False,Valido,True,False,True,Nulo,NaT,False,28438089
97,22917811-3,1970-04-19,Masculino,83353,Extranjera,Antonio Misael,De Avila,Romero,None,2024-02-15 20:19:41.791000+00:00,...,False,True,Nulo,False,True,False,Valido,1970-04-19,True,22917811
98,22917811-3,1970-04-19,Masculino,83353,Extranjera,Antonio Misael,De Avila,Romero,None,2025-07-17 15:08:16.701000+00:00,...,True,False,Valido,False,True,True,Duplicado,NaT,False,22917811
99,18631264-3,1994-03-01,Masculino,710725,Chilena,Mauricio Fernando,Bastías,Cortés,None,2025-03-26 03:28:35.956000+00:00,...,False,True,Nulo,True,False,True,Nulo,1994-03-01,True,18631264


In [28]:
df_antecedentes_personales = df_antecedentes_personales[['id', 'fecha_nacimiento','sexo','nro_registro','nacionalidad','nombres','apellido_paterno',	'apellido_materno_x']]
def limpiar_valor(valor):
    """Elimina caracteres especiales, convierte a minúsculas, y elimina tildes"""
    # Eliminar tildes y acentos
    valor_sin_tildes = unicodedata.normalize('NFKD', valor).encode('ASCII', 'ignore').decode('ASCII')
    # Eliminar caracteres especiales
    valor_limpio = re.sub(r'[^\w\s]', '', valor_sin_tildes)
    return valor_limpio.replace(' ', '_').lower()
df_antecedentes_personales['nacionalidad'] = df_antecedentes_personales['nacionalidad'].astype(str).apply(limpiar_valor)

# Convertir el DataFrame en la lista de diccionarios
df_antecedentes_personales['fecha_hoy'] = pd.to_datetime('today').strftime('%Y-%m-%d')

antecedentes_personales = df_antecedentes_personales.to_dict(orient='records')

contactos = antecedentes_personales
contactos

/tmp/ipython-input-28-1509188438.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_antecedentes_personales['nacionalidad'] = df_antecedentes_personales['nacionalidad'].astype(str).apply(limpiar_valor)
/tmp/ipython-input-28-1509188438.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_antecedentes_personales['fecha_hoy'] = pd.to_datetime('today').strftime('%Y-%m-%d')


[{'id': '39469',
  'fecha_nacimiento': '1988-11-19',
  'sexo': 'Femenino',
  'nro_registro': '362722',
  'nacionalidad': 'chilena',
  'nombres': 'Isabel Soledad',
  'apellido_paterno': 'Fuentes',
  'apellido_materno_x': 'Zambrano',
  'fecha_hoy': '2025-07-17'},
 {'id': '137655262555',
  'fecha_nacimiento': '1988-11-19',
  'sexo': 'Femenino',
  'nro_registro': '362722',
  'nacionalidad': 'chilena',
  'nombres': 'Isabel Soledad',
  'apellido_paterno': 'Fuentes',
  'apellido_materno_x': 'Zambrano',
  'fecha_hoy': '2025-07-17'},
 {'id': '11546051',
  'fecha_nacimiento': '1989-08-11',
  'sexo': 'Femenino',
  'nro_registro': '580765',
  'nacionalidad': 'extranjera',
  'nombres': 'Ana Maria',
  'apellido_paterno': 'Campos',
  'apellido_materno_x': 'Romero',
  'fecha_hoy': '2025-07-17'},
 {'id': '137642051333',
  'fecha_nacimiento': '1995-09-20',
  'sexo': 'Femenino',
  'nro_registro': '657127',
  'nacionalidad': 'chilena',
  'nombres': 'Teresita María',
  'apellido_paterno': 'Fuenzalida',
  '

In [ ]:
import requests
import os

# --- Mensajes de Log para este bloque ---
log_file.write("\n--- Bloque Final: Actualizando atributos de contacto en HubSpot ---\n")

# CORREGIDO: Se usa la variable de entorno para la llave de API
ACCESS_TOKEN = os.getenv('HUBSPOT_API_KEY')
base_url = 'https://api.hubapi.com/crm/v3/objects/contacts'
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}',
    'Content-Type': 'application/json'
}

# Función para actualizar los atributos de un contacto
def actualizar_contacto(contacto_id, fecha_nacimiento, sexo, nro_registro, nacionalidad, nombres, apellido_paterno, apellido_materno_x, fecha_hoy):
    url = f'{base_url}/{contacto_id}'
    data = {
        'properties': {
            'date_of_birth': fecha_nacimiento,
            'gender': sexo,
            'registro_superintendencia': nro_registro,
            'nacionalidad': nacionalidad,
            'firstname': nombres,
            'lastname': apellido_paterno,
            'apellido_materno': apellido_materno_x,
            'fecha_act_super': fecha_hoy
        }
    }
    response = requests.patch(url, json=data, headers=headers)
    return response

# Asumo que la variable 'contactos' es una lista de diccionarios que ya existe de una celda anterior
total_contactos = len(contactos)
log_file.write(f"Iniciando la actualización de {total_contactos} contactos...\n")

# Iterar sobre la lista de contactos y actualizar los atributos
for index, contacto in enumerate(contactos):
    response = actualizar_contacto(
        contacto['id'], 
        contacto['fecha_nacimiento'], 
        contacto['sexo'],
        contacto['nro_registro'],
        contacto['nacionalidad'],
        contacto['nombres'],
        contacto['apellido_paterno'],
        contacto['apellido_materno_x'],
        contacto['fecha_hoy']
    )

    # CORREGIDO: Se escribe el resultado de cada actualización en el log
    if response.status_code == 200:
        log_file.write(f"({index + 1}/{total_contactos}) Éxito: Contacto {contacto['id']} actualizado.\n")
    else:
        log_file.write(f"!!! ERROR al actualizar contacto {contacto['id']}: {response.status_code} - {response.text}\n")

log_file.write("--- Proceso de actualización de contactos finalizado ---\n")

# --- MUY IMPORTANTE ---
# Esta línea debe ser la ÚLTIMA en todo tu notebook para asegurar que el log se guarde.


Contacto 39469 actualizado exitosamente
Contacto 137655262555 actualizado exitosamente
Contacto 11546051 actualizado exitosamente
Contacto 137642051333 actualizado exitosamente
Contacto 113940880902 actualizado exitosamente
Contacto 30036 actualizado exitosamente
Contacto 23024351 actualizado exitosamente
Contacto 81184476742 actualizado exitosamente
Contacto 28506951 actualizado exitosamente
Contacto 108919543181 actualizado exitosamente
Contacto 20443 actualizado exitosamente
Contacto 16975343620 actualizado exitosamente
Contacto 28162051 actualizado exitosamente
Contacto 27857390431 actualizado exitosamente
Contacto 27593496142 actualizado exitosamente
Contacto 108875453725 actualizado exitosamente
Contacto 61046551069 actualizado exitosamente
Contacto 61046553167 actualizado exitosamente
Contacto 61174649439 actualizado exitosamente
Contacto 68541402794 actualizado exitosamente
Contacto 114236657685 actualizado exitosamente
Contacto 72039767965 actualizado exitosamente
Contacto 148

In [ ]:
mem_end = process.memory_info().rss
end = time.time()

log_file.write("\n====================================================\n")
log_file.write("== FIN DEL SCRIPT ==\n")
log_file.write(f"Tiempo total: {end - start:.2f} segundos\n")
log_file.write(f"Memoria total usada: {(mem_end - mem_start)/1024**2:.2f} MB\n")
log_file.write("====================================================\n")

# --- MUY IMPORTANTE ---
# Esta línea debe ser la ÚLTIMA en todo tu notebook para asegurar que el log se guarde.
log_file.close()

Tiempo total: 673.64 segundos
Memoria total usada: 455.47 MB
